In [7]:
import pandas as pd
import numpy as np
from IPython.display import display
import torch

from config import *

from src.datafeed.downstream import get_fx_data

In [2]:
def style_time_series(df, n_tail=6, mult=1.0, precision=2):
    _res = df.tail(n_tail).mul(mult).rename_axis(index="date").reset_index()\
        .style.format({"date": "{:%Y-%m-%d}"}, precision=precision).hide()
    return _res

In [18]:
# load data
data = get_fx_data()

For each currency, I calculate 1-month excess returns $rx$ in a forward-looking manner, that is, $rx_t = \frac{S_{t+1m}}{F_t}$. Note that the returns are observed daily, which increases the number of data points twenty-fold compared to the case when monthly returns are observed monthly.

In [22]:
rx = data["excess_returns"]

print("1-month forward-looking excess returns, in %")
display(style_time_series(rx.loc[["2020-12-01"]], n_tail=1, mult=100))

1-month forward-looking excess returns, in %


date,aud,cad,chf,dkk,eur,gbp,jpy,nok,nzd,sek
2020-12-01,3.95,1.21,1.92,1.41,1.35,1.06,1.08,2.88,2.65,2.62


For each currency, define the date-$t$ term structure history (TSH) as the $T \times M$ matrix containing the history of forward prices for $M$ tenors over the past $T$ periods.

In [21]:
# display term struct history
tsh = data["term_structure_history"]

print("6-day term structure history of aud on 2020-12-01")
display(
    style_time_series(
        tsh.loc[:"2020-12-01", "aud"], 
        precision=4
    ).background_gradient(cmap='Reds')
)

6-day term structure history of aud on 2020-12-01


date,spot,1m,2m,3m,6m,9m,12m,2y
2020-11-24,0.7361,0.7363,0.7365,0.7366,0.7369,0.7372,0.7375,0.7384
2020-11-25,0.7365,0.7367,0.7368,0.7370,0.7373,0.7376,0.7379,0.7387
2020-11-26,0.7362,0.7364,0.7365,0.7367,0.7370,0.7373,0.7376,0.7384
2020-11-27,0.7387,0.7389,0.7391,0.7392,0.7395,0.7398,0.7401,0.7408
2020-11-30,0.7344,0.7347,0.7348,0.7350,0.7353,0.7356,0.7358,0.7363
2020-12-01,0.7371,0.7374,0.7376,0.7377,0.7381,0.7384,0.7386,0.7392


Let's also define the normalized term structure history NTSH as follows: divide all values by the date-$t$ spot price, take the log and 'annualize' all values except the spot column:

In [25]:
annualizer = pd.Series({"spot": 1,
                        "1m"  : 12/1,
                        "2m"  : 12/2,
                        "3m"  : 12/3,
                        "6m"  : 12/6,
                        "9m"  : 12/9,
                        "12m" : 12/12,
                        "2y"  : 12/24})

def normalize_term_structure(ts):
    # div by spot, take log, annualize
    _res = ts\
        .div(ts.iloc[-1].loc["spot"])\
        .pipe(np.log)\
        .mul(annualizer, axis=1)
    return _res

In [26]:
print("normalized 6-day term structure history of aud on 2020-12-01")
display(
    style_time_series(
        normalize_term_structure(
            data["term_structure_history"].loc[:"2020-12-01", "aud"].tail(6)
        ), 
        precision=4
    ).background_gradient(cmap='Reds')
)

normalized 6-day term structure history of aud on 2020-12-01


date,spot,1m,2m,3m,6m,9m,12m,2y
2020-11-24,-0.0014,-0.0136,-0.0053,-0.0028,-0.0004,0.0003,0.0006,0.0009
2020-11-25,-0.0008,-0.0072,-0.0021,-0.0008,0.0006,0.0010,0.0011,0.0011
2020-11-26,-0.0012,-0.0121,-0.0046,-0.0024,-0.0003,0.0004,0.0006,0.0009
2020-11-27,0.0022,0.0297,0.0159,0.0112,0.0066,0.0049,0.0040,0.0025
2020-11-30,-0.0037,-0.0395,-0.0185,-0.0117,-0.0049,-0.0027,-0.0017,-0.0005
2020-12-01,0.0000,0.0046,0.0037,0.0032,0.0026,0.0023,0.0021,0.0014


This normalized matrix encodes many signals commonly used to construct FX strategies. For instance, a carry trade signal can be extracted by simply observing the most recent value in column '1m', the 5-day momentum signal &ndash; by observing the value in column 'spot' five days from today, a skewness signal &ndash; by subtracting the most recent value in column '12m' from that in columne '1m', and so on. In fact, any signal that is a linear transformation of the NTSH can be extracted with a suitable convolution. For instance, the following convolution extracts the 1-month carry signal:

In [33]:
example_ntsh = normalize_term_structure(
    data["term_structure_history"].loc[:"2020-12-01", "aud"].tail(6)
)
lookback, n_tenors = example_ntsh.shape